In [13]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk

In [14]:
df = pd.read_csv("api_method_desc.csv")

In [15]:
df

,Unnamed: 0,api_method,api_method_desc
0,0,javax.swing.AbstractAction.getValue,Gets the Object associated with the specified ...
1,1,javax.swing.AbstractAction.putValue,Sets the Value associated with the specified key.
2,2,javax.swing.AbstractAction.isEnabled,Returns true if the action is enabled.
3,3,javax.swing.AbstractAction.setEnabled,Sets whether the Action is enabled. The defaul...
4,4,javax.swing.AbstractAction.getKeys,Returns an array of Objects which are keys for...
...,...,...,...
31731,31731,org.omg.PortableServer._ServantActivatorStub.e...,This operation is invoked whenever a servant f...
31732,31732,org.omg.PortableServer._ServantActivatorStub._ids,Description copied from class: ObjectImpl
31733,31733,org.omg.PortableServer._ServantLocatorStub.pre...,This operations is used to get a servant that ...
31734,31734,org.omg.PortableServer._ServantLocatorStub.pos...,This operation is invoked whenener a servant c...


In [16]:
api_method_desc_list = df["api_method_desc"].to_list()

In [17]:
# for s in api_method_desc_list:
#     print(s.split(".")[0])
desc_list = sorted([s.split(". ")[0] for s in api_method_desc_list])

In [18]:
desc_list = [s for s in desc_list if len(s.split(" "))<=14]

In [19]:
desc_list = [s for s in desc_list if "and" not in s]
desc_list = [s for s in desc_list if "which " not in s]
desc_list = [s for s in desc_list if "whether" not in s]
desc_list = [s for s in desc_list if "when" not in s]
desc_list = [s for s in desc_list if "A " not in s]
desc_list = [s for s in desc_list if "such as " not in s]

desc_list = [s for s in desc_list if "Deprecated" not in s]

In [20]:
lint_ = sorted(list(set(desc_list)))

In [21]:
from nltk import ngrams
import collections
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
trigrams_list=[]
corpus = " ".join(lint_)
# xgrams = ngrams(corpus.split(), 3)
xgrams = ngrams(corpus.split(), 7)
collections.Counter(xgrams).most_common()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


[(('Sets', 'the', 'designated', 'parameter', 'to', 'the', 'given'), 42),
 (('color', 'of', 'the', 'current', 'theme', 'Returns', 'the'), 36),
 (('Returns', 'the', 'hash', 'code', 'value', 'for', 'this'), 35),
 (('primitive', 'conversion.', 'Returns', 'the', 'value', 'of', 'this'), 35),
 (('Returns', 'a', 'copy', 'of', 'this', 'LocalDateTime', 'with'), 25),
 (('a', 'copy', 'of', 'this', 'LocalDateTime', 'with', 'the'), 25),
 (('Returns', 'a', 'copy', 'of', 'this', 'OffsetDateTime', 'with'), 25),
 (('a', 'copy', 'of', 'this', 'OffsetDateTime', 'with', 'the'), 25),
 (('Returns', 'a', 'copy', 'of', 'this', 'ZonedDateTime', 'with'), 25),
 (('a', 'copy', 'of', 'this', 'ZonedDateTime', 'with', 'the'), 25),
 (('Gets', 'the', 'value', 'of', 'the', 'specified', 'field'), 24),
 (('after', 'a', 'widening', 'primitive', 'conversion.', 'Returns', 'the'),
  24),
 (('value', 'Updates', 'the', 'designated', 'column', 'with', 'a'), 23),
 (('the', 'value', 'of', 'the', 'specified', 'field', 'from'), 22),

In [22]:

verbs=[]
for line in lint_:
    doc = nlp(line)
    text=""
    pos=""
    tag=""
    dep=""
    
    for idx,token in enumerate(doc):
#         text+=token.text+" "
#         pos+=token.pos_+" "
        text= [token.norm_ for token in doc if token.pos_ in ["VERB"] ]
    if len(text):
        verbs.append(text[0])


In [23]:
# import collections
# collections.Counter(verbs).most_common()



In [24]:
import gensim 
from gensim.models import KeyedVectors
from nltk.corpus import brown
import nltk 

nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [25]:
from nltk.data import find
import spacy
nlp = spacy.load('en')

import collections
import glob
import os
import pandas as pd
from itertools import combinations
import random

# Importing drive method from colab for accessing google drive
from google.colab import drive
# Mounting drive
# This will require authentication : Follow the steps as guided
#drive.mount('/content/drive')

drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [26]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [44]:
uverbs = set(verbs)
# for v in uverbs:
#   print(v)
uverbs = ' '.join(uverbs)
all_tokens = nlp(uverbs)

roots = []
for token in all_tokens:
  roots.append(token.lemma_)

df = pd.DataFrame(index=roots, columns=roots,dtype=float)
n = len(roots)
for i in range(n):
  for j in range(n):
    if i != j:
      try:
        df.iloc[i,j] = model.similarity(roots[i], roots[j])
      except KeyError:
        pass
    else:
      df.iloc[i,j] = 0

print(df)




                        adapt       set  ...    reduce  jformattedtextfield
adapt                0.000000  0.143496  ...  0.217744                  NaN
set                  0.143496  0.000000  ...  0.019457                  NaN
mark                 0.063154  0.234681  ...  0.017435                  NaN
assign               0.217830  0.197863  ...  0.265478                  NaN
describe             0.230801  0.080274  ...  0.094902                  NaN
...                       ...       ...  ...       ...                  ...
pass                 0.166469  0.117324  ...  0.077468                  NaN
apply                0.283334  0.187442  ...  0.258344                  NaN
paint                0.072408  0.115822  ...  0.083664                  NaN
reduce               0.217744  0.019457  ...  0.000000                  NaN
jformattedtextfield       NaN       NaN  ...       NaN                  0.0

[929 rows x 929 columns]


In [104]:
indices = df.index.to_list()
a = {}
for i, v in enumerate(indices):
  for j, b in enumerate(indices):
    if i != j:
      temp = list(df.iloc[i,:])
      temp, indices = zip(*sorted(zip(temp, indices), reverse=True))
      a[v] = indices[0:9]
a

{'abort': ('find',
  'rectangle2d',
  'back',
  'byte',
  'detect',
  'imply',
  'search',
  'lowercase',
  'pass'),
 'aborted': ('getter',
  'associate',
  'prepende',
  'describe',
  'acquire',
  'ostream',
  'cancel',
  'sequence',
  'paint'),
 'aborting': ('replay',
  'register',
  'complete',
  'locate',
  'remain',
  'sleep',
  'designate',
  'evaluate',
  'block'),
 'accept': ('extend',
  'render',
  'array',
  'offset',
  'prepend',
  'extract',
  'resolve',
  'lay',
  'resolve'),
 'access': ('write',
  'remove',
  'synchronize',
  'build',
  'find',
  'resolve',
  'unset',
  'represent',
  'supported'),
 'accessing': ('evaluate',
  'remap',
  'enforce',
  'ascend',
  'nextsible',
  'read',
  'come',
  'prepare',
  'know'),
 'accord': ('deliver',
  'follow',
  'inform',
  'round',
  'search',
  'filterinputstream.skip',
  'detect',
  'fetch',
  'scaling'),
 'accumulate': ('draw',
  'see',
  'edit',
  'remap',
  'run',
  'prepare',
  'consider',
  'decode',
  'consider'),
 'accu

In [105]:
import json
newdf = json.dumps(a, sort_keys=True, indent=4, separators=(',', ': '))
with open('verbList.json', 'w+') as f:
    f.write(newdf)